In [1]:
from z3 import *
from random import randint
def declare(i): # declara as variaveis de cada estado #
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['m'] = BitVec('m'+str(i),16)
    state['n'] = BitVec('n'+str(i),16)
    state['r'] = BitVec('r'+str(i),16)
    state['x'] = BitVec('x'+str(i),16)
    state['y'] = BitVec('y'+str(i),16)
    return state

def init(state): # inicia o primeiro estado #
    return And(state['m']==randint(0,20),state['n']==randint(0,20),
               state['r']==0,state['x']==state['m'],
               state['y']==state['n'],state['pc']==0)

def trans(curr,prox): # define as tranciçoes possiveis #
    # define igualdade dos valores do estado anterior para o seguinte #
    ti=And(prox['m']==curr['m'],prox['n']==curr['n'],prox['r']==curr['r'],prox['x']==curr['x'],prox['y']==curr['y'])
    
    # as possiveis tranciçoes #
    t1=And(curr['pc']==0,prox['pc']==1,curr['y']<=0,ti)
    t2=And(curr['pc']==0,prox['pc']==0,curr['y']>0,prox['m']==curr['m'],prox['n']==curr['n'],prox['r']==curr['r']+curr['x'],
           prox['x']==(curr['x']<<1),prox['y']==((curr['y']-1)>>1),Not(curr['y']==0))
    t3=And(curr['pc']==0,prox['pc']==0,curr['y']>0,prox['m']==curr['m'],prox['n']==curr['n'],prox['r']==curr['r'],
           prox['x']==(curr['x']<<1),prox['y']==(curr['y']>>1),curr['y']==0)
    t4=And(curr['pc']==1,prox['pc']==1,ti)
    return Or(t1,t2,t3,t4)

In [3]:
def kinduction_always(declare,init,trans,inv,k):
    # completar
    s=Solver()
    state=[declare(i) for i in range(k)]
    s.add(init(state[0]))
    s.add(And([trans(state[i],state[i+1]) for i in range(k-1)]))
    s.add(Or([Not(inv(state[i])) for i in range(k)]))
    if s.check()==sat:
        print("O invariante é falso num dos primeiros",k,"estados")
        return
    assert(s.check()==unsat)
    s=Solver()
    state=[declare(i) for i in range(k+1)]
    s.add(init(state[0]))
    s.add(And([trans(state[i],state[i+1]) for i in range(k)]))
    s.add(And([inv(state[i]) for i in range(k)]))
    s.add(Not(inv(state[k])))
    if s.check()==sat:
        print("Não é possivel provar o invariante com",k,"indução")
        return
    assert(s.check()==unsat)
    print("Propriedade valida!")

In [18]:
def variante(state):
    return state['y']

def termina(state):
    return (state['y'] == 0)

def variante_positivo(state):
    return variante(state)>=0

kinduction_always(declare,init,trans,variante_positivo,3)

def variante_decrescente(state):
    nextstate=declare(-1)
    return ForAll(list(nextstate.values()),
                  Implies(trans(state,nextstate),Or(variante(nextstate)< variante(state),variante(nextstate)==0)))

kinduction_always(declare,init,trans,variante_decrescente,3)

def variante_util(state):
    return Implies(variante(state)==0,termina(state))

kinduction_always(declare,init,trans,variante_util,3)

Propriedade valida!
Propriedade valida!
Propriedade valida!
